In [50]:
import random
import math
import sys
from operator import attrgetter
sys.path.append("../")
from random import Random
from VectorIndividual import Individual
from fitnessFunc import fitnessFunc
import copy
#Import individuals and set values
randomSeed = 1234
uniprng=Random()
uniprng.seed(randomSeed)
normprng=Random()
normprng.seed(randomSeed+101)

#Set parameters
Individual.minSigma=1e-100
Individual.maxSigma=1
Individual.learningRate=1
Individual.minLimit=-5.12
Individual.maxLimit=5.12
Individual.uniprng= uniprng
Individual.normprng= normprng
Individual.fitFunc= fitnessFunc
Individual.vectorLength = 2

# Initialization

In [51]:
populationSize = 3
crossoverFraction = 0.4
population = [Individual() for i in range(populationSize)]

for individual in population:
    print(individual)


--------------------------
[INFO]Float Vector
x_vec:
[4.776484205487502, -0.6068981844443888]
Fit:
39.354634865089864


--------------------------
[INFO]Float Vector
x_vec:
[4.208393855479032, 4.4981145330049435]
Fit:
55.35839435709722


--------------------------
[INFO]Float Vector
x_vec:
[1.7568100504369673, -4.26047255718826]
Fit:
31.467785870341803




# CrossOver

In [52]:
children = []

indexList1=list(range(len(population)))
indexList2=list(range(len(population)))
uniprng.shuffle(indexList1)
uniprng.shuffle(indexList2)
if crossoverFraction == 1.0:
    for index1,index2 in zip(indexList1,indexList2):
        childx = population[index1].crossover(population[index2])
        children.append(childx)
else:
    for index1,index2 in zip(indexList1,indexList2):
        rn=uniprng.random()
        if rn < crossoverFraction:
            childx = population[index1].crossover(population[index2])
            children.append(childx)


# Childrem Compete then Replace some parents.
for childx in children:
    for idx, parent in enumerate(population):
        if childx.fit < parent.fit:
            population[idx] = childx
            break

for individual in population:
    print(individual)


--------------------------
[INFO]Float Vector
x_vec:
[4.776484205487502, -0.6068981844443888]
Fit:
39.354634865089864


--------------------------
[INFO]Float Vector
x_vec:
[4.208393855479032, 4.4981145330049435]
Fit:
55.35839435709722


--------------------------
[INFO]Float Vector
x_vec:
[1.7568100504369673, -4.26047255718826]
Fit:
31.467785870341803




# Conduct Tournament

In [53]:
mode = "MIN"
# binary tournament
indexList1=list(range(len(population)))
indexList2=list(range(len(population)))
uniprng.shuffle(indexList1)
uniprng.shuffle(indexList2)
# do not allow population competition
for i in range(len(population)):
    if indexList1[i] == indexList2[i]:
        temp=indexList2[i]
        if i == 0:
            indexList2[i]=indexList2[-1]
            indexList2[-1]=temp
        else:
            indexList2[i]=indexList2[i-1]
            indexList2[i-1]=temp
#compete
newPop=[]
for index1,index2 in zip(indexList1,indexList2):
    if mode == 'MIN':
        if population[index1].fit < population[index2].fit:
            newPop.append(copy.deepcopy(population[index1]))
        elif population[index1].fit > population[index2].fit:
            newPop.append(copy.deepcopy(population[index2]))
        else:
            rn= uniprng.random()
            if rn > 0.5:
                newPop.append(copy.deepcopy(population[index1]))
            else:
                newPop.append(copy.deepcopy(population[index2]))
    else:
        if population[index1].fit > population[index2].fit:
            newPop.append(copy.deepcopy(population[index1]))
        elif population[index1].fit < population[index2].fit:
            newPop.append(copy.deepcopy(population[index2]))
        else:
            rn= uniprng.random()
            if rn > 0.5:
                newPop.append(copy.deepcopy(population[index1]))
            else:
                newPop.append(copy.deepcopy(population[index2]))

# overwrite old pop with newPop
population=newPop


In [54]:
for individual in population:
    print(individual)

--------------------------
[INFO]Float Vector
x_vec:
[1.7568100504369673, -4.26047255718826]
Fit:
31.467785870341803


--------------------------
[INFO]Float Vector
x_vec:
[1.7568100504369673, -4.26047255718826]
Fit:
31.467785870341803


--------------------------
[INFO]Float Vector
x_vec:
[4.776484205487502, -0.6068981844443888]
Fit:
39.354634865089864




# truncateSelect

In [55]:
newPopSize = len(population)
#sort by fitness
if mode == "MIN":
    population.sort(key=attrgetter('fit'),reverse=False)
else:
    population.sort(key=attrgetter('fit'),reverse=True)
    
#then truncate the bottom
population=population[:newPopSize]

In [56]:
for individual in population:
    print(individual)

--------------------------
[INFO]Float Vector
x_vec:
[1.7568100504369673, -4.26047255718826]
Fit:
31.467785870341803


--------------------------
[INFO]Float Vector
x_vec:
[1.7568100504369673, -4.26047255718826]
Fit:
31.467785870341803


--------------------------
[INFO]Float Vector
x_vec:
[4.776484205487502, -0.6068981844443888]
Fit:
39.354634865089864




# Putting in all together

In [57]:
#
# Population.py
#
#

import copy
import math
from operator import attrgetter
from Individual import *

class Population:
    """
    Population
    """
    uniprng=None
    crossoverFraction=None
    mode   = 'MIN'

    def __init__(self, populationSize):
        """
        Population constructor
        """
        self.population = [Individual() for i in range(populationSize)]
        

    def __len__(self):
        return len(self.population)

    def __getitem__(self,key):
        return self.population[key]

    def __setitem__(self,key,newValue):
        self.population[key]=newValue

    def copy(self):
        return copy.deepcopy(self)

    def evaluateFitness(self):
        for individual in self.population: individual.evaluateFitness()

    def mutate(self):
        for individual in self.population:
            individual.mutate()

    def crossover(self):
        children = []

        indexList1=list(range(len(self.population)))
        indexList2=list(range(len(self.population)))
        self.uniprng.shuffle(indexList1)
        self.uniprng.shuffle(indexList2)
        if self.crossoverFraction == 1.0:
            for index1,index2 in zip(indexList1,indexList2):
                childx = self.population[index1].crossover(self.population[index2])
                children.append(childx)
        else:
            for index1,index2 in zip(indexList1,indexList2):
                rn=uniprng.random()
                if rn < crossoverFraction:
                    childx = self.population[index1].crossover(self.population[index2])
                    children.append(childx)

        # Childrem Compete then Replace some parents.
        for childx in children:
            for idx, parent in enumerate(self.population):
                if childx.fit < parent.fit:
                    self.population[idx] = childx
                    break
    
            
    def conductTournament(self):
        # binary tournament
        indexList1=list(range(len(self.population)))
        indexList2=list(range(len(self.population)))
        uniprng.shuffle(indexList1)
        uniprng.shuffle(indexList2)
        
        # do not allow population competition
        for i in range(len(self.population)):
            if indexList1[i] == indexList2[i]:
                temp=indexList2[i]
                if i == 0:
                    indexList2[i]=indexList2[-1]
                    indexList2[-1]=temp
                else:
                    indexList2[i]=indexList2[i-1]
                    indexList2[i-1]=temp
        #compete
        newPop=[]
        for index1,index2 in zip(indexList1,indexList2):
            if self.mode == 'MIN':
                if self.population[index1].fit < self.population[index2].fit:
                    newPop.append(copy.deepcopy(self.population[index1]))
                elif self.population[index1].fit > self.population[index2].fit:
                    newPop.append(copy.deepcopy(self.population[index2]))
                else:
                    rn= uniprng.random()
                    if rn > 0.5:
                        newPop.append(copy.deepcopy(population[index1]))
                    else:
                        newPop.append(copy.deepcopy(self.population[index2]))
            else:
                if self.population[index1].fit > self.population[index2].fit:
                    newPop.append(copy.deepcopy(self.population[index1]))
                elif self.population[index1].fit < self.population[index2].fit:
                    newPop.append(copy.deepcopy(self.population[index2]))
                else:
                    rn= uniprng.random()
                    if rn > 0.5:
                        newPop.append(copy.deepcopy(self.population[index1]))
                    else:
                        newPop.append(copy.deepcopy(self.population[index2]))

        # overwrite old pop with newPop
        self.population=newPop

    def combinePops(self,otherPop):
        self.population.extend(otherPop.population)

    def truncateSelect(self,newPopSize):
        newPopSize = len(population)
        #sort by fitness
        if mode == "MIN":
            population.sort(key=attrgetter('fit'),reverse=False)
        else:
            population.sort(key=attrgetter('fit'),reverse=True)

        #then truncate the bottom
        population=population[:newPopSize]

    def __str__(self):
        print("--------------------------\n[INFO]Vector Population")
        for individual in self.population:
            print(individual)
        return '\n'


In [59]:
#Import individuals and set values
randomSeed = 1234
uniprng=Random()
uniprng.seed(randomSeed)
normprng=Random()
normprng.seed(randomSeed+101)

#Set parameters
Individual.minSigma=1e-100
Individual.maxSigma=1
Individual.learningRate=1
Individual.minLimit=-5.12
Individual.maxLimit=5.12
Individual.uniprng= uniprng
Individual.normprng= normprng
Individual.fitFunc= fitnessFunc
Individual.vectorLength = 2

Population.uniprng=uniprng
Population.crossoverFraction=crossoverFraction
Population.mode = 'MAX'
generationCount = 20

#create initial Population (random initialization)
population=Population(populationSize)

print(population)

#evolution main loop
for i in range(generationCount):
    #create initial offspring population by copying parent pop
    offspring=population.copy()
    #select mating pool
    offspring.conductTournament()
    #perform crossover
    offspring.crossover()
    #random mutation
    offspring.mutate()
    #update fitness values
    offspring.evaluateFitness()
    #survivor selection: elitist truncation using parents+offspring
    population.combinePops(offspring)
    population.truncateSelect(populationSize)

TypeError: object of type 'float' has no len()